In [1]:
# Instalação da biblioteca PuLP, caso ainda não esteja instalada
!pip install pulp

import pulp

# ---------------------------------------------------------------------------
# DADOS EXTRAÍDOS DO DOCUMENTO (Tabelas 1, 2 e 3 do trabalho de 1995)
# ---------------------------------------------------------------------------

# Estratos: E1, E2, E3, E4, E5
estratos = ['E1', 'E2', 'E3', 'E4', 'E5']

# Coeficientes para as Situações I e II (Tabela 1) [cite: 71, 461]
dados_sit_1_2 = {
    'area_max': {'E1': 21.66, 'E2': 14.17, 'E3': 11.33, 'E4': 6.34, 'E5': 7.53},
    'renda_liquida': {'E1': 404.40, 'E2': 286.50, 'E3': 183.30, 'E4': 102.57, 'E5': 127.00},
    'rend_lenha': {'E1': 50.57, 'E2': 53.08, 'E3': 43.09, 'E4': 40.71, 'E5': 38.69},
    'mao_de_obra': {'E1': 67.5, 'E2': 68.3, 'E3': 58.0, 'E4': 56.5, 'E5': 48.6},
    'transporte': {'E1': 6.7, 'E2': 6.3, 'E3': 4.8, 'E4': 4.5, 'E5': 5.6}
}
# O valor da mão de obra para E1 na Situação I e II é 87.5 no Apêndice [cite: 717], mas 67.5 na Tabela 1[cite: 461].
# Usaremos o valor da Tabela 1, pois o do Apêndice parece ser um erro de digitação.
# Corrigindo com base na Tabela 1:
dados_sit_1_2['mao_de_obra']['E1'] = 67.5

# Coeficientes para as Situações III e IV (Tabela 2) [cite: 492]
dados_sit_3_4 = {
    'area_max': {'E1': 21.66, 'E2': 14.17, 'E3': 11.33, 'E4': 6.34, 'E5': 7.53},
    'renda_liquida': {'E1': 486.5, 'E2': 301.2, 'E3': 195.4, 'E4': 110.7, 'E5': 131.2},
    'rend_lenha': {'E1': 49.63, 'E2': 51.50, 'E3': 39.70, 'E4': 36.70, 'E5': 34.90},
    'rend_estacas': {'E1': 170, 'E2': 275, 'E3': 105, 'E4': 68, 'E5': 71}, # Ordem invertida na tabela original [cite: 492]
    'mao_de_obra': {'E1': 67.5, 'E2': 68.3, 'E3': 58.0, 'E4': 56.5, 'E5': 48.6},
    'transporte': {'E1': 6.0, 'E2': 6.5, 'E3': 4.3, 'E4': 3.9, 'E5': 5.4} # Ordem invertida na tabela original
}

# Coeficientes para a Situação V (Tabela 3) [cite: 509]
dados_sit_5 = {
    'area_max': {'E1': 21.66, 'E2': 14.17, 'E3': 11.33, 'E4': 6.34, 'E5': 7.53},
    'renda_liquida': {'E1': 498.30, 'E2': 314.10, 'E3': 207.80, 'E4': 223.50, 'E5': 142.60},
    'rend_lenha': {'E1': 49.63, 'E2': 51.50, 'E3': 39.70, 'E4': 36.70, 'E5': 34.90},
    'rend_estacas': {'E1': 170, 'E2': 275, 'E3': 105, 'E4': 68, 'E5': 71},
    'transporte': {'E1': 6.0, 'E2': 6.5, 'E3': 4.3, 'E4': 3.9, 'E5': 5.4}
}


def resolver_situacao_original(nome_situacao, dados, restricoes):
    """
    Formula e resolve um problema de programação linear para as situações originais.
    """
    # Criação do problema de maximização
    problema = pulp.LpProblem(nome_situacao, pulp.LpMaximize)

    # Variáveis de decisão (área em hectares de cada estrato)
    E = pulp.LpVariable.dicts("Area", estratos, lowBound=0, cat='Continuous')

    # Função Objetivo: Maximizar a Renda Líquida
    problema += pulp.lpSum([dados['renda_liquida'][i] * E[i] for i in estratos]), "Renda_Liquida_Total"

    # Restrições
    # 1. Limite de área por estrato
    for i in estratos:
        problema += E[i] <= dados['area_max'][i], f"Limite_Area_{i}"

    # 2. Restrições específicas da situação
    if 'sl_min' in restricoes:
        problema += pulp.lpSum([dados['rend_lenha'][i] * E[i] for i in estratos]) >= restricoes['sl_min'], "Suprimento_Lenha_Min"
    if 'mo_max' in restricoes:
        problema += pulp.lpSum([dados['mao_de_obra'][i] * E[i] for i in estratos]) <= restricoes['mo_max'], "Mao_de_Obra_Max"
    if 'nc_max' in restricoes:
        problema += pulp.lpSum([dados['transporte'][i] * E[i] for i in estratos]) <= restricoes['nc_max'], "Transporte_Max"
    if 'em_min' in restricoes:
        problema += pulp.lpSum([dados['rend_estacas'][i] * E[i] for i in estratos]) >= restricoes['em_min'], "Suprimento_Estacas_Min"

    # Resolver o problema
    problema.solve()

    # Exibir resultados
    print(f"--- RESULTADOS: {nome_situacao} ---")
    print(f"Status da Solução: {pulp.LpStatus[problema.status]}")
    print(f"Valor da Função Objetivo (Renda Líquida): US$ {pulp.value(problema.objective):.2f}")
    print("Área a ser explorada por estrato (ha):")
    for i in estratos:
        if E[i].varValue > 0:
            print(f"  - {i}: {E[i].varValue:.2f}")
    print("-" * 35 + "\n")


def resolver_modelo_multiobjetivo(lambda_val, dados, restricoes, sensibilidade_ambiental, var_type='Integer'):
    """
    Formula e resolve o modelo estendido com múltiplos objetivos (econômico e ambiental)
    e variáveis inteiras.
    """
    nome_problema = f"Modelo_Multiobjetivo_Lambda_{lambda_val:.2f}"
    problema = pulp.LpProblem(nome_problema, pulp.LpMaximize)

    # Variáveis de decisão
    cat = 'Integer' if var_type == 'Integer' else 'Continuous'
    E = pulp.LpVariable.dicts("Area", estratos, lowBound=0, cat=cat)

    # Função Objetivo Ponderada
    # Objetivo Econômico
    z_econ = pulp.lpSum([dados['renda_liquida'][i] * E[i] for i in estratos])
    # Objetivo Ambiental (minimizar impacto, logo, subtrai-se na maximização)
    z_ambiental = pulp.lpSum([sensibilidade_ambiental[i] * E[i] for i in estratos])

    problema += lambda_val * z_econ - (1 - lambda_val) * z_ambiental, "Objetivo_Ponderado"

    # Restrições (iguais à situação IV, mas com adições ambientais)
    for i in estratos:
        problema += E[i] <= dados['area_max'][i], f"Limite_Area_{i}"

    problema += pulp.lpSum([dados['rend_lenha'][i] * E[i] for i in estratos]) >= restricoes['sl_min'], "Suprimento_Lenha_Min"
    problema += pulp.lpSum([dados['mao_de_obra'][i] * E[i] for i in estratos]) <= restricoes['mo_max'], "Mao_de_Obra_Max"
    problema += pulp.lpSum([dados['rend_estacas'][i] * E[i] for i in estratos]) >= restricoes['em_min'], "Suprimento_Estacas_Min"

    # Novas Restrições Ambientais
    area_sensivel = ['E1', 'E2'] # Caatinga Arbórea Fechada
    problema += pulp.lpSum([E[i] for i in area_sensivel]) <= restricoes['area_sensivel_max'], "Limite_Area_Sensivel"

    area_total_fazenda = 460 # ha [cite: 328]
    prop_preservacao = 0.20 # 20% de reserva
    problema += pulp.lpSum([E[i] for i in estratos]) <= (1 - prop_preservacao) * area_total_fazenda, "Cota_Reserva_Legal"

    # Resolver o problema
    problema.solve()

    # Coletar e retornar resultados para análise de trade-off
    if pulp.LpStatus[problema.status] == 'Optimal':
        lucro = sum(dados['renda_liquida'][i] * E[i].varValue for i in estratos)
        impacto = sum(sensibilidade_ambiental[i] * E[i].varValue for i in estratos)
        return {
            'lambda': lambda_val,
            'lucro_total': lucro,
            'impacto_ambiental_total': impacto,
            'areas': {i: E[i].varValue for i in estratos if E[i].varValue > 0}
        }
    return None

# --- ANÁLISE DAS SITUAÇÕES ORIGINAIS ---

print("=" * 50)
print("REVISÃO E SOLUÇÃO DOS MODELOS ORIGINAIS (1995)")
print("=" * 50 + "\n")

# Situação I [cite: 455, 456, 457]
restricoes_sit_1 = {'sl_min': 2300, 'mo_max': 3500, 'nc_max': 310}
resolver_situacao_original("Situacao_I_Corrigida", dados_sit_1_2, restricoes_sit_1)

# Situação II [cite: 471]
restricoes_sit_2 = {'sl_min': 2600, 'mo_max': 3700, 'nc_max': 350}
resolver_situacao_original("Situacao_II", dados_sit_1_2, restricoes_sit_2)

# Situação III [cite: 476, 477, 478, 479]
restricoes_sit_3 = {'sl_min': 1700, 'mo_max': 3500, 'em_min': 9250, 'nc_max': 450}
resolver_situacao_original("Situacao_III_Corrigida", dados_sit_3_4, restricoes_sit_3)

# Situação IV [cite: 485, 486, 487, 488]
restricoes_sit_4 = {'sl_min': 2500, 'mo_max': 4500, 'em_min': 8000}
resolver_situacao_original("Situacao_IV", dados_sit_3_4, restricoes_sit_4)

# Situação V [cite: 502, 503, 504, 505]
restricoes_sit_5 = {'sl_min': 2500, 'em_min': 8000, 'nc_max': 400}
resolver_situacao_original("Situacao_V", dados_sit_5, restricoes_sit_5)


# --- ANÁLISE DO MODELO MULTIOBJETIVO ESTENDIDO ---

print("\n" + "=" * 50)
print("ANÁLISE DO MODELO MULTIOBJETIVO ESTENDIDO")
print("=" * 50)
print("Analisando o trade-off entre Lucro (Econômico) e Impacto (Ambiental)\n")

# Coeficientes de sensibilidade ambiental (hipotéticos, para ilustração)
# Estratos de caatinga fechada (E1, E2) têm maior peso de impacto
sensibilidade_ambiental = {'E1': 1.0, 'E2': 0.9, 'E3': 0.6, 'E4': 0.5, 'E5': 0.4}

# Usando restrições da Situação IV como base para o modelo estendido
restricoes_base_multiobjetivo = {
    'sl_min': 2500,
    'mo_max': 4500,
    'em_min': 8000,
    'area_sensivel_max': 25, # Limite hipotético de 25 ha para a área de caatinga fechada
}

resultados_tradeoff = []
# Varrer diferentes valores de lambda para construir a fronteira de Pareto
lambdas = [1.0, 0.9, 0.7, 0.5, 0.3, 0.1, 0.0]

for l in lambdas:
    resultado = resolver_modelo_multiobjetivo(l, dados_sit_3_4, restricoes_base_multiobjetivo, sensibilidade_ambiental)
    if resultado:
        resultados_tradeoff.append(resultado)

# Exibir tabela de trade-off
print("--- Fronteira de Trade-off (Lucro vs. Impacto Ambiental) ---")
print("-" * 75)
print(f"{'Lambda':<10} | {'Lucro Total (US$)':<20} | {'Impacto Ambiental':<22} | {'Áreas (ha)':<20}")
print("-" * 75)
for res in resultados_tradeoff:
    lucro_str = f"US$ {res['lucro_total']:.2f}"
    impacto_str = f"{res['impacto_ambiental_total']:.2f}"
    areas_str = ", ".join([f"{k}:{v}" for k, v in res['areas'].items()])
    print(f"{res['lambda']:<10.2f} | {lucro_str:<20} | {impacto_str:<22} | {areas_str}")
print("-" * 75)
print("\nObservação: Lambda=1.0 foca 100% no lucro. Lambda=0.0 foca 100% em minimizar o impacto ambiental.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 37.8 MB/s eta 0:00:00
REVISÃO E SOLUÇÃO DOS MODELOS ORIGINAIS (1995)

--- RESULTADOS: Situacao_I_Corrigida ---
Status da Solução: Optimal
Valor da Função Objetivo (Renda Líquida): US$ 15379.54
Área a ser explorada por estrato (ha):
  - E1: 21.66
  - E2: 14.17
  - E3: 11.33
  - E4: 4.72
-----------------------------------

--- RESULTADOS: Situacao_II ---
Status da Solução: Optimal
Valor da Função Objetivo (Renda Líquida): US$ 16286.67
Área a ser explorada por estrato (ha):
  - E1: 21.66
  - E2: 14.17
  - E3: 11.33
  - E4: 4.68
  - E5: 7.17
-----------------------------------

--- RESULTADOS: Situacao_III_Corrigida ---
Status da Solução: Optimal
Valor da Função Objetivo (Renda Líquida): US$ 18099.58
Área a ser explorada por estrato (ha):
  - E1: 21.66
  - E2: 14.17
  - E3: 11.33
  - E4: 0.83
  - E5: 7.53
-----------------------------------

--- RESULTADOS: Situacao_IV ---
Status da Solução: Optimal
Valor da Função Objetivo (Renda L